# Building an ETL Pipeline

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime, date, time, timedelta
import tweepy  
import numpy as np 
import pyodbc
from sqlalchemy import create_engine
import urllib
import re
import geopandas as gpd

###  Consumer and Access details

In [2]:
# Consumer:
CONSUMER_KEY    = 'uXfKji3NyWLDEGI3gXxx3tBNc'
CONSUMER_SECRET = 'eiuC2xhRfbmbY2gSplZOcL4dPSSUiqmB8SbLk6eGuZ2DaBaYQj'

# Access:
ACCESS_TOKEN  = '1219607153163603968-oOHm1E2uXjyBiBOkkKNExfAwvU4dVd'
ACCESS_SECRET = '3dZ7jyldTV1m1RxoVPv8VEOT92m2OwEPpF1Syj0j2Mu95'

In [3]:
# API's setup:
def twitter_setup():
    """
    Utility function to setup the Twitter's API
    with access and consumer keys from Twitter.
    """

    # Authentication and access using keys:
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

    # Return API with authentication:
    api = tweepy.API(auth, timeout=1000)
    return api

# Function 1: Twitter scraper

In [4]:
# Function for twitter scraper
def twitter_df():
    ## Code Here
    '''  Scrapes "Eskom_SA" tweets directly from Twitter.Return a dataframe with only tweets and dates.
        
        Returns :
        A dataframe which contain tweets and dates column
        '''
    
    api = twitter_setup()
    all_tweets = []
    tweets = api.user_timeline(screen_name='ESKOM_SA', 
                           # 200 is the maximum allowed count
                           count=200,
                           include_rts = False,
                           # Necessary to keep full_text 
                           # otherwise only the first 140 words are extracted
                           tweet_mode = 'extended'
                           )
    all_tweets.extend(tweets)
  
    outtweets = [[tweet.full_text.encode("utf-8").decode("utf-8"),tweet.created_at] 
              for idx,tweet in enumerate(all_tweets)]
    df = pd.DataFrame(outtweets,columns=['Tweets','Date'])
    df.set_index('Tweets')
    return df
twitter_df()

,Tweets,Date
0,Report electricity theft anonymously to Eskom ...,2020-03-10 12:38:00
1,#POWERALERT 2\n\nDate: 10 March 2020\n\nStage ...,2020-03-10 11:50:38
2,#Chooseday: Choose to do the right thing and p...,2020-03-10 10:06:00
3,Prepaid pricing is based on usage/consumption ...,2020-03-10 08:30:00
4,"@TSIKALIRA Hi, pls DM us your contact details ...",2020-03-10 06:32:50
5,#POWERALERT 1\nDate: 10 March 2020\n\nStage 2 ...,2020-03-10 05:37:17
6,Protect your electrical appliances by unpluggi...,2020-03-10 04:30:00
7,@Jeandre_S @CityTshwane @SABCNewsOnline @SABCR...,2020-03-09 17:32:41
8,@Antifreez777za @SikonathiM Please see our tim...,2020-03-09 17:26:55
9,@Zwanedefault Work of scam artists,2020-03-09 17:25:20


In [5]:
# Creating function which extracts municipality and hashtags
def extract_municipality_hashtags(df):
    ## Code Here
    
    ''' Function which extract municpality and hashtags from twitter and insert them into new twitter dataframe.
        Returns: A dataframe which contain tweets, date, municiplaity and hashtags columns'''
    
    mun_dict = {'@CityofCTAlerts' : 'Cape Town',
                '@CityPowerJhb' : 'Johannesburg',
                '@eThekwiniM' : 'eThekwini' ,
                '@EMMInfo' : 'Ekurhuleni',
                '@centlecutility' : 'Mangaung',
                '@NMBmunicipality' : 'Nelson Mandela Bay',
                '@CityTshwane' : 'Tshwane'}
    
    handles = list(mun_dict.keys())
    
    df['municipality'] = df['Tweets'].str.extract('({})'.format('|'.join(handles)), expand=False).fillna(np.nan)
    df['municipality'] = df['municipality'].map(mun_dict)
    
    df['hashtags'] = df['Tweets'].str.findall(r'#.*?(?=\s|$)')
    df['hashtags'] = df['hashtags'].apply(lambda x: np.nan if len(x)==0 else [x.lower() for x in x])
    
    return df
extract_municipality_hashtags(twitter_df())

,Tweets,Date,municipality,hashtags
0,Report electricity theft anonymously to Eskom ...,2020-03-10 12:38:00,NaN,NaN
1,#POWERALERT 2\n\nDate: 10 March 2020\n\nStage ...,2020-03-10 11:50:38,NaN,[#poweralert]
2,#Chooseday: Choose to do the right thing and p...,2020-03-10 10:06:00,NaN,[#chooseday:]
3,Prepaid pricing is based on usage/consumption ...,2020-03-10 08:30:00,NaN,[#useelectricitysmartly]
4,"@TSIKALIRA Hi, pls DM us your contact details ...",2020-03-10 06:32:50,NaN,NaN
5,#POWERALERT 1\nDate: 10 March 2020\n\nStage 2 ...,2020-03-10 05:37:17,NaN,[#poweralert]
6,Protect your electrical appliances by unpluggi...,2020-03-10 04:30:00,NaN,[#loadshedding]
7,@Jeandre_S @CityTshwane @SABCNewsOnline @SABCR...,2020-03-09 17:32:41,Tshwane,NaN
8,@Antifreez777za @SikonathiM Please see our tim...,2020-03-09 17:26:55,NaN,NaN
9,@Zwanedefault Work of scam artists,2020-03-09 17:25:20,NaN,NaN


In [6]:
# Defining the extracted dataframe
df = extract_municipality_hashtags(twitter_df())

### Function 3: Updating SQL Database with pyODBC

In [7]:
# Convert original hashtags datatype to string 
df['hashtags'] =(df['hashtags']).astype(str)

In [8]:
# Establish SQL server connection
''' Connecting to SQL server  '''

# Open database connection    
conn = pyodbc.connect(driver='{SQL Server}' ,
                     host='MANDLA63\SQLEXPRESS' ,
                     database='gather_eskom' ,
                     trusted_connection='true',
                     user='sa')
# Read query in the dataframe:
twitter_table= pd.read_sql_query('select * from dbo.twitter_table',conn)

In [9]:
# Create twitter_table on SQL server

def pyodbc_twitter(connection, df, twitter_table):
    
    ## Code Here
    ''' Creating a twitter_table on SQL server, once this code is executed for second time an existing table in SQL datebase will be replaced '''
    
    params = urllib.parse.quote_plus("DRIVER={SQL Server};SERVER=MANDLA63\SQLEXPRESS;DATABASE=gather_eskom;UID=sa;trusted_connection='true'")
    engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
    df.to_sql('twitter_table', con=engine, if_exists='replace')
    
    ''' Initializing connection, update  and store twitter_table on SQL '''
    
    conn = pyodbc.connect(driver='{SQL Server}' ,
                     host='MANDLA63\SQLEXPRESS' ,
                     database='gather_eskom' ,
                     trusted_connection='true',
                      user='sa')
    twitter_table= pd.read_sql_query('select * from dbo.twitter_table',conn)
    return None
pyodbc_twitter(conn, df, twitter_table)

In [10]:
# Check that twitter_table is in MS SQL Database ,then display five rows
df = pd.read_sql_query('select * from dbo.twitter_table',conn)

# View first five rows from the twitter_table in SQL database 
df.head()

,index,Tweets,Date,municipality,hashtags
0,0,Report electricity theft anonymously to Eskom ...,2020-03-10 12:38:00,None,nan
1,1,#POWERALERT 2\n\nDate: 10 March 2020\n\nStage ...,2020-03-10 11:50:38,None,['#poweralert']
2,2,#Chooseday: Choose to do the right thing and p...,2020-03-10 10:06:00,None,['#chooseday:']
3,3,Prepaid pricing is based on usage/consumption ...,2020-03-10 08:30:00,None,['#useelectricitysmartly']
4,4,"@TSIKALIRA Hi, pls DM us your contact details ...",2020-03-10 06:32:50,None,nan


### Extra Eskom data

In [11]:
existing_powerlines =gpd.read_file('C:/Users/smesh/Desktop/Explore Data Science/ANALY/eskom/ANALYSE/FINA/Shapefiles/Existing_Lines.shp')
existing_powerlines.head()

,OBJECTID_1,PL_NO,LINE_START,LINE_END,LINE_NUMBE,DESIGN_VOL,NOMINAL_VO,LINE_STATU,ID,SUB_CAT,LABEL,geometry
0,1,251,PELLY,WARMBAD,1.0,275.0,275.0,EXISTING,251,275kv_line,PELLY WARMBAD 1,"LINESTRING Z (28.29331 -25.41602 0.00000, 28.3..."
1,2,256,PRINCESS,TAUNUS,1.0,275.0,275.0,EXISTING,256,275kv_line,PRINCESS TAUNUS 1,"LINESTRING Z (27.84598 -26.17453 0.00000, 27.8..."
2,3,257,PRINCESS,WESTGATE,1.0,275.0,275.0,EXISTING,257,275kv_line,PRINCESS WESTGATE 1,"LINESTRING Z (27.75609 -26.14799 0.00000, 27.7..."
3,4,259,TABOR,WITKOP,1.0,275.0,275.0,EXISTING,259,275kv_line,TABOR WITKOP 1,"LINESTRING Z (29.78247 -23.37569 0.00000, 29.7..."
4,5,270,WARMBAD,WITKOP,1.0,275.0,275.0,EXISTING,270,275kv_line,WARMBAD WITKOP 1,"LINESTRING Z (29.35498 -24.04278 0.00000, 29.3..."


In [12]:
existing_substations = gpd.read_file('C:/Users/smesh/Desktop/Explore Data Science/ANALY/eskom/ANALYSE/FINA/Shapefiles/Existing_Substations.shp')
existing_substations.head()

,OBJECTID_2,ID,status,In_New_GIS,Substation,UnlockingM,NetworkStr,Province,Phasing,Generation,DurationFo,EIACostRan,ServitudeR,PreEngCost,EstimatedC,geometry
0,0,OLIEN,EXISTING,None,Olien,1100.0,"400kV S/S, 3x500MVA 400/132kV, 70km Olien - U...",Northern Cape,4.0,Renewable,6-8 years,10.0,100.0,10.0,1570.0,POINT (23.62378 -28.33209)
1,0,VUYANI,EXISTING,None,Vuyani,0.0,None,Eastern Cape,1.0,Renewable,2 years,0.0,0.0,1.5,0.0,POINT (28.81351 -31.64190)
2,0,NAMA,EXISTING,None,Nama,263.0,2x250 220/132kV,Northern Cape,3.0,Renewable,4-5 years,0.0,1.0,3.0,246.0,POINT (17.88402 -29.63235)
3,0,PAULPUTS,EXISTING,None,Paulputs,250.0,"400kV S/S, Replace 220/132kV 125MVA with 250MV...",Northern Cape,4.0,Renewable,6-8 years,5.0,5.0,5.0,725.0,POINT (19.57587 -28.86015)
4,0,AGGENEIS,EXISTING,None,Aggeneis,220.0,"New 132kV S/S, 2x250MVA 400/132kV",Northern Cape,2.0,Renewable,3-4 years,0.0,0.0,5.0,346.0,POINT (18.80338 -29.29758)


In [13]:
planned_lines = gpd.read_file('C:/Users/smesh/Desktop/Explore Data Science/ANALY/eskom/ANALYSE/FINA/Shapefiles/Planned_Lines.shp')
planned_lines.head()

,OBJECTID_1,GP_Project,SchemeName,PrjctName,Voltage,ExpectedCO,Province,Shape_Leng,geometry
0,1,GPP0200,Medupi Integration Phase 2A:Mogwase,Medupi-Ngwedi 1st 400 kV line,400.0,2019,North West,2.011287,MULTILINESTRING Z ((27.41137 -23.79716 0.00000...
1,2,GPP0264,Kimberley Strengthening Phase 3,Hermes-Mookodi (Vryburg) 1st 400 kV line,400.0,2026,North West,2.335985,"LINESTRING Z (24.75060 -27.01237 0.00000, 24.7..."
2,3,GPP0378,Medupi Integration Phase 2A:Mogwase,Medupi-Ngwedi 1st 765 kV line (Energised at 40...,400.0,2019,North West,1.999838,MULTILINESTRING Z ((27.41072 -23.79758 0.00000...
3,4,GPP0597,Watershed (Backbone) Strengthening Phase 3,Pluto-Mahikeng 400 kV,400.0,2024,North West,1.849203,"LINESTRING Z (25.77343 -25.87489 0.00000, 25.7..."
4,5,GPP0598,Watershed (Backbone) Strengthening Phase 3,Mookodi-Mahikeng 400 kV,400.0,2029,North West,1.849034,"LINESTRING Z (24.74374 -27.01234 0.00000, 24.7..."


In [14]:
planned_substations = gpd.read_file('C:/Users/smesh/Desktop/Explore Data Science/ANALY/eskom/ANALYSE/FINA/Shapefiles/Planned_Substations.shp')
planned_substations.head()

,ID,status,New_Year,Substation,UnlockingM,NetworkStr,Province,Phasing,Generation,DurationFo,EIACostRan,ServitudeR,PreEngCost,EstimatedC,geometry
0,BLOUWATER,New,None,Blouwater/Aurora B,1325.0,"New 400kV S/S, 4x500MVA 400/132kV, 2x30km Auro...",Western Cape,4.0,Renewable,6-8 years,10.0,50.0,10.0,1326.0,POINT (17.99907 -32.98123)
1,KAPPA,New,2016,Koruson(Kappa ),276.0,"132kV S/S, 2x250MVA 400/132kV",Western Cape,3.0,Renewable,4-5 years,0.0,1.0,5.0,346.0,POINT (20.00036 -33.11002)
2,GAMMA,New,2017,Gamma,974.0,"400kV S/S, 3x500MVA 400/132kV , 2x4km Hydra- D...",Northern Cape,3.0,Renewable,4-5 years,0.0,5.0,12.0,533.0,POINT (23.40462 -31.67934)
3,LAMBDA,New,None,Lambda,121.2,"New 88kV S/S, 2x315MVA 400/88kV",Mpumalanga,2.0,Renewable,3-4 years,0.0,0.0,5.0,346.0,POINT (29.87637 -27.09416)
4,UPINGTON SOLAR PARK,New,2016,Upington Solar Park,1000.0,"3x500MVA 400/132kV, 2x150km Upington-Aries 1 ...",Northern Cape,1.0,Renewable,2 years,0.0,0.0,5.0,2869.0,POINT (21.13198 -28.54347)


In [15]:
## Renewable Energy Independent Power Producer
reipp = gpd.read_file('C:/Users/smesh/Desktop/Explore Data Science/ANALY/eskom/ANALYSE/FINA/Shapefiles/RE_IPP_1_to_4b.shp')
reipp.head()

,OBJECTID,ID,IPP_Ref_No,Project_Na,Project_Co,Province,Closest_To,RE_IPP_Prg,Technology,Technolo_1,MEC_MW_All,X,Y,geometry
0,1,1.0,RE_BM_0261_002,Ngodwana Energy,Ngodwana Energy (RF) Proprietary Limited,Mpumalanga,Nelspruit,RE IPP 4,Biomass,BM,25.00,30.661944,-25.566667,POINT (30.66194 -25.56667)
1,2,2.0,RE_OW_0046_003,Golden Valley Wind,Amstilite (Pty) Ltd,Eastern Cape,Cookhouse,RE IPP 4,Onshore Wind,OW,117.72,25.960158,-32.903786,POINT (25.96016 -32.90379)
2,3,3.0,RE_OW_0024_008,Karusa Wind Farm,ACED Renewables Hidden Valley (Pty) Ltd,Northern Cape,Sutherland,RE IPP 4,Onshore Wind,OW,139.80,20.634825,-32.869853,POINT (20.63483 -32.86985)
3,4,4.0,RE_OW_0024_020,Nxuba Wind Farm,Nxuba Wind Farm (RF) (Pty) Ltd,Eastern Cape,Cookhouse,RE IPP 4,Onshore Wind,OW,138.90,25.966842,-32.737669,POINT (25.96684 -32.73767)
4,5,5.0,RE_OW_0154_001,Oyster Bay Wind Farm,Oyster Bay Wind Farm,Eastern Cape,Humansdorp,RE IPP 4,Onshore Wind,OW,140.00,24.648889,-34.116389,POINT (24.64889 -34.11639)


In [16]:
supp_areas = gpd.read_file('C:/Users/smesh/Desktop/Explore Data Science/ANALY/eskom/ANALYSE/FINA/Shapefiles/Supply_Areas2022_Steady_State_Limit.shp')
supp_areas.head()

,OBJECTID,Name,CLNLimit,Shape_Leng,Shape_Area,Supply_Are,CLNLimitMD,geometry
0,1,Bloemfontein,2600,22.541133,7.152140,Bloemfontein,1500,"MULTIPOLYGON (((27.18587 -28.37976, 27.25967 -..."
1,2,Carletonville,2650,13.196033,4.668323,Carletonville,2650,"POLYGON ((26.63462 -25.06591, 26.68515 -25.354..."
2,3,East London,550,10.730411,4.488472,East London,1050,"POLYGON ((29.66569 -31.54845, 29.66429 -31.549..."
3,4,East Rand,4500,3.265573,0.373556,East Rand,2050,"POLYGON ((28.67024 -25.90069, 28.69726 -25.956..."
4,5,Empangeni,3700,4.635284,0.764575,Empangeni,4000,"POLYGON ((31.56731 -29.17039, 31.49153 -29.114..."
